# Philosophy Chat Bot

data set pdf links: https://www.kaggle.com/datasets/kouroshalizadeh/history-of-philosophy/data


https://www.infobooks.org/pdfview/7529-eastern-philosophy-jsrl-narayana-moorty/
https://www.infobooks.org/authors/classic/plato-books/#Republic
https://www.infobooks.org/book/on-youth-old-age-life-and-death-and-respiration-aristotle/
https://www.infobooks.org/pdfview/7225-the-communist-manifesto-karl-marx/


related sources:
https://www.kaggle.com/code/gpreda/rag-using-llama3-langchain-and-chromadb
https://www.kaggle.com/code/vanvalkenberg/nlp-what-the-philosopher-said/notebook

https://www.youtube.com/watch?v=luFHMtaw9pk&ab_channel=DavidBU
https://www.youtube.com/watch?v=2TJxpyO3ei4&t=2s&ab_channel=pixegami
https://www.youtube.com/watch?v=tcqEUSNCn8I&ab_channel=pixegami
https://www.youtube.com/watch?v=Ylz779Op9Pw&ab_channel=ShawTalebi
https://www.youtube.com/watch?v=au2WVVGUvc8&t=307s&ab_channel=LiamOttley

### Download dataset

In [2]:
%%python -m pip install --upgrade pip
%pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
# Install kagglehub package
%pip install kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("kouroshalizadeh/history-of-philosophy")
path = path + "/philosophy_data.csv"

print("Path to dataset files:", path)

Note: you may need to restart the kernel to use updated packages.


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/codespace/.cache/kagglehub/datasets/kouroshalizadeh/history-of-philosophy/versions/3/philosophy_data.csv


### Settin up Ollama

In [4]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
!sudo apt-get update
!sudo apt-get install -y pciutils
!sudo apt-get install -y lshw

from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()


Reading package lists... Done
Building dependency tree       
Reading state information... Done
pciutils is already the newest version (1:3.6.4-1ubuntu0.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%#######                                                          23.1%                    27.9%        49.2%.5%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:2 https://dl.yarnpkg.com/debian stable InRelease                     

### Install Dependencies

In [5]:
! pip install langchain langchain_community langchain_chroma 
! pip install -qU "langchain-chroma>=0.1.2"

2024/11/24 18:32:34 routes.go:1197: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/codespace/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://*] OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES: http_proxy: https_proxy: no_proxy:]"
time=2024-11-24T18:32:34.108Z level=INFO source=ima

In [6]:
pip install pysqlite3-binary  

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import sys

BASE_DIR = os.path.dirname(sys.executable)
print(BASE_DIR)

/home/codespace/.python/current/bin


In [8]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}

### Import Libraries

In [9]:
import bs4
from langchain import hub
# from langchain_chroma import Chroma #make sure to have sqlite3 version 3.35.0 or higher
# from chromadb.config import Settings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma

### Load models

In [10]:
# Pull models from ollama
!ollama pull llama3.1
!ollama pull nomic-embed-text

[GIN] 2024/11/24 - 18:32:40 | 200 |      28.863µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ [GIN] 2024/11/24 - 18:32:41 | 200 |  449.115637ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 
[GIN] 2024/11/24 - 18:32:41 | 200 |      24.385µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ [GIN] 2024/11/24 - 18:32:42 | 200 |  663.790548ms |       127.0.0.1 | POST     "/ap

In [11]:
# Load LLM
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.1")

/tmp/ipykernel_4321/242480205.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1")


In [12]:
# Load Embeddings - convert text into vector representations
embeddings = OllamaEmbeddings(model="nomic-embed-text")

/tmp/ipykernel_4321/146061546.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


### Load documents

In [18]:
import random

loader = CSVLoader(
    file_path=path, 
    source_column="author",
    content_columns=["sentence_str"],
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
    },
    metadata_columns=["school", "title","original_publication_date"],
    )

# Take only 100,000 random documents from the docs list
docs = loader.load()
docs = random.sample(docs, 100000)

for record in docs[:2]:
    print(record)

page_content='sentence_str: So I should imagine.' metadata={'source': 'Plato', 'row': 32991, 'school': 'plato', 'title': 'Plato - Complete Works', 'original_publication_date': '-350'}
page_content='sentence_str: And again, you think that those who do what's unjust are happy, so long as they don't pay what is due.' metadata={'source': 'Plato', 'row': 18783, 'school': 'plato', 'title': 'Plato - Complete Works', 'original_publication_date': '-350'}


### Splitting Documents and store

In [19]:
# Create a RecursiveCharacterTextSplitter object with specified chunk size and overlap
# chunk_size=1000: The maximum size of each chunk (in characters) to split the document into.
# chunk_overlap=200: The number of characters that should overlap between consecutive chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into smaller chunks using the splitter
split_documents = text_splitter.split_documents(docs)

print(len(split_documents))

100032


In [20]:
print(split_documents[20].page_content)

sentence_str: So if we say that Ion is equally clever about Homer and the other poets, we'll make no mistake.


In [21]:
# pip install --upgrade chromadb

In [ ]:
# Create a vector store using the Chroma library from the split documents
# Chroma is a vector database that stores document embeddings.
# The `from_documents` method takes the list of split documents and generates embeddings for them.
vectorstore = Chroma.from_documents(
    documents=split_documents,  # The list of split documents
    collection_name="rag-chroma",  # Name of the collection to store in the vector store
    embedding=embeddings,  # The embedding model used to convert the documents into vector representations
    # persist_directory="chroma_db"
)

# Create a retriever from the vector store
# The retriever will allow you to query the vector store and retrieve relevant documents based on vector similarity.
retriever = vectorstore.as_retriever()

[GIN] 2024/11/24 - 20:53:34 | 200 |  152.017198ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:34 | 200 |   383.27455ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:35 | 200 |  905.087512ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:36 | 200 |  688.680904ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:36 | 200 |  556.955256ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:36 | 200 |  330.452994ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:37 | 200 |  396.124605ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:37 | 200 |  306.254436ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:38 | 200 |  424.807804ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:38 | 200 |  299.424351ms |             ::1 | POST     "/api/embeddings"
[GIN] 2024/11/24 - 20:53:38 | 

time=2024-11-24T21:01:01.395Z level=INFO source=routes.go:509 msg="embedding generation failed: do embedding request: Post \"http://127.0.0.1:37697/embedding\": context canceled"


[GIN] 2024/11/24 - 21:01:01 | 500 |  393.298865ms |             ::1 | POST     "/api/embeddings"


KeyboardInterrupt: 

time=2024-11-24T21:01:01.831Z level=INFO source=.:0 msg="http: superfluous response.WriteHeader call from main.(*Server).embeddings (runner.go:790)"


### Prompt Construction

In [ ]:
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

### Chain Construction

In [ ]:
rag_chain = (
    # Define inputs: context from retriever, question passed through unchanged.
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt  # Format inputs with the prompt
    | llm  # Generate response using the LLM
    | StrOutputParser()  # Parse output as a string
)